In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from bioinf_common.plotting import corrplot

In [ ]:
sns.set_context('talk')

# Parameters

In [ ]:
result_files = snakemake.input

out_dir = Path(snakemake.output.out_dir)

# Load data

In [ ]:
df_list = []
for fname in result_files:
    tmp = pd.read_csv(fname)
    
    _, tool, _, _ = fname.split('/')
    tmp['tool'] = tool
    
    df_list.append(tmp)
df = pd.concat(df_list, ignore_index=True)

In [ ]:
df.head()

# Enrichment correlation between tools

In [ ]:
df_wide = df.pivot(index='term', columns='tool', values='p_value')
df_wide.head()

In [ ]:
g = corrplot(
    df_wide,
    corr_method='spearman',
    diag_kws=dict(
        kde=False,
        bins=np.linspace(df_wide.values.ravel().min(), df_wide.values.ravel().max(), 50)
    ),
    lower_kws=dict(rasterized=True),
    diag_sharey=False, height=3
)

g.savefig(out_dir / 'pvalue_scatterplots.pdf')